In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=ebee662bf3b578e2892174814f8e8245cd64f247512d9c3b5b969a59dbb5fdce
  Stored in directory: /Users/jameswilliams/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [10]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [11]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [12]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1       False_Positive
2       False_Positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_Positive
6987    False_Positive
6988         CANDIDATE
6989    False_Positive
6990    False_Positive
Name: koi_disposition, Length: 6991, dtype: object

# Select your features (columns)

In [13]:
# Set features. This will also be used as your x values.
selected_features_df = df.drop(columns=["koi_disposition"])

# Create a Train Test Split

Use `koi_disposition` for the y values

In [14]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [15]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4002,0,0,1,0,99.673478,3.463000e-04,-3.463000e-04,219.334830,0.002300,-0.002300,...,-148,4.777,0.040,-0.027,0.492,0.026,-0.027,293.05801,45.248821,15.801
4246,0,1,0,0,0.592244,9.000000e-08,-9.000000e-08,131.654831,0.000124,-0.000124,...,-146,4.664,0.056,-0.032,0.591,0.045,-0.045,290.28094,45.464260,15.653
548,0,1,1,0,9.991625,5.360000e-06,-5.360000e-06,137.447816,0.000445,-0.000445,...,-176,4.338,0.153,-0.187,1.096,0.309,-0.206,301.04239,45.022888,14.039
3953,0,1,0,0,178.412990,3.100000e-05,-3.100000e-05,218.225235,0.000127,-0.000127,...,-134,4.346,0.084,-0.126,1.148,0.202,-0.124,288.32785,38.627621,13.944
2362,0,0,0,0,45.294223,5.600000e-05,-5.600000e-05,138.678725,0.000987,-0.000987,...,-68,4.347,0.030,-0.030,1.044,0.057,-0.042,285.67938,50.241299,10.961


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [16]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [18]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical.shape

(5243, 3)

# Train the Model



In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [20]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [22]:
from keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 1s - loss: 0.5689 - accuracy: 0.7105
Epoch 2/60
164/164 - 0s - loss: 0.3691 - accuracy: 0.8072
Epoch 3/60
164/164 - 0s - loss: 0.3614 - accuracy: 0.8079
Epoch 4/60
164/164 - 0s - loss: 0.3433 - accuracy: 0.8280
Epoch 5/60
164/164 - 0s - loss: 0.3338 - accuracy: 0.8405
Epoch 6/60
164/164 - 0s - loss: 0.3272 - accuracy: 0.8440
Epoch 7/60
164/164 - 0s - loss: 0.3266 - accuracy: 0.8402
Epoch 8/60
164/164 - 0s - loss: 0.3161 - accuracy: 0.8499
Epoch 9/60
164/164 - 0s - loss: 0.3246 - accuracy: 0.8423
Epoch 10/60
164/164 - 0s - loss: 0.3110 - accuracy: 0.8570
Epoch 11/60
164/164 - 0s - loss: 0.3106 - accuracy: 0.8570
Epoch 12/60
164/164 - 0s - loss: 0.3055 - accuracy: 0.8566
Epoch 13/60
164/164 - 0s - loss: 0.3001 - accuracy: 0.8648
Epoch 14/60
164/164 - 0s - loss: 0.3029 - accuracy: 0.8615
Epoch 15/60
164/164 - 0s - loss: 0.2930 - accuracy: 0.8674
Epoch 16/60
164/164 - 0s - loss: 0.2902 - accuracy: 0.8705
Epoch 17/60
164/164 - 0s - loss: 0.2918 - accuracy: 0.8671
Epoch 

Epoch 45/60
164/164 - 0s - loss: 0.2497 - accuracy: 0.8911
Epoch 46/60
164/164 - 0s - loss: 0.2533 - accuracy: 0.8888
Epoch 47/60
164/164 - 0s - loss: 0.2536 - accuracy: 0.8911
Epoch 48/60
164/164 - 0s - loss: 0.2528 - accuracy: 0.8879
Epoch 49/60
164/164 - 0s - loss: 0.2468 - accuracy: 0.8909
Epoch 50/60
164/164 - 0s - loss: 0.2441 - accuracy: 0.8951
Epoch 51/60
164/164 - 0s - loss: 0.2487 - accuracy: 0.8903
Epoch 52/60
164/164 - 0s - loss: 0.2445 - accuracy: 0.8930
Epoch 53/60
164/164 - 0s - loss: 0.2473 - accuracy: 0.8928
Epoch 54/60
164/164 - 0s - loss: 0.2438 - accuracy: 0.8932
Epoch 55/60
164/164 - 0s - loss: 0.2427 - accuracy: 0.8951
Epoch 56/60
164/164 - 0s - loss: 0.2420 - accuracy: 0.8949
Epoch 57/60
164/164 - 0s - loss: 0.2427 - accuracy: 0.8949
Epoch 58/60
164/164 - 0s - loss: 0.2420 - accuracy: 0.8947
Epoch 59/60
164/164 - 0s - loss: 0.2482 - accuracy: 0.8894
Epoch 60/60
164/164 - 0s - loss: 0.2423 - accuracy: 0.8940


In [23]:
#Quantify trained model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2775 - accuracy: 0.8770
Normal Neural Network - Loss: 0.27752333879470825, Accuracy: 0.8770022988319397


In [25]:
#Make predictions
import numpy as np
encoded_predictions = np.argmax(model.predict(X_test_scaled[:5]), axis=-1)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [26]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CANDIDATE' 'False_Positive' 'False_Positive' 'CANDIDATE'
 'False_Positive']
Actual Labels: ['CANDIDATE', 'False_Positive', 'False_Positive', 'CANDIDATE', 'False_Positive']


# Save the Model

In [30]:
model.save("deep_learning_model.h5")